In [106]:
import pandas as pd
from sodapy import Socrata
import os
from scipy.spatial import cKDTree
import endpoints
import geopandas as gpd
import numpy as np 

In [107]:
# Extract environment variables
APP_TOKEN = os.getenv('APP_TOKEN')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')

In [108]:
# Get data with app token to avoid data limits on call 
client = Socrata(endpoints.CLIENT_IDENTIFIER, APP_TOKEN, username=USERNAME, password=PASSWORD)
squirrel_census_results = client.get(endpoints.SQUIRREL_DATASET_IDENTIFIER)
tree_census_results = client.get(endpoints.TREE_CENSUS_DATASET_IDENTIFIER)

In [109]:
# Convert data to Pandas dataframe
squirrel_census_results_df = pd.DataFrame.from_records(squirrel_census_results)
squirrel_column_names=squirrel_census_results_df.columns.values.tolist()

tree_census_results_df = pd.DataFrame.from_records(tree_census_results)
tree_column_names=tree_census_results_df.columns.values.tolist()

In [110]:
print(tree_census_results_df.columns.values.tolist())

['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other', 'address', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude', 'longitude', 'x_sp', 'y_sp', 'council_district', 'census_tract', 'bin', 'bbl']


In [111]:
# Squirrels per hectare
squirrels_per_hectare=squirrel_census_results_df.groupby('hectare')['unique_squirrel_id'].count().reset_index(name="squirrel_count")
squirrels_per_hectare.to_csv("squirrels_per_hectare.csv", index=False)

In [112]:
# Proportion of squirrels eating aboveground vs on the ground 
eating_counts = (squirrel_census_results_df[squirrel_census_results_df['eating'] == True].groupby('above_ground_sighter')['eating'].count().reset_index(name='eating_squirrel_count'))
eating_counts.to_csv("num_squirrels_eating_per_elevation_aboveground.csv", index=False)

In [113]:
# Number of activities squirrel is doing at once
squirrel_census_results_df['activity_score'] = squirrel_census_results_df[['running', 'chasing', 'climbing', 'eating', 'foraging']].sum(axis=1)

In [114]:
# Parse date field so its more human readable
squirrel_census_results_df['date'] = pd.to_datetime(squirrel_census_results_df['date'], format='%m%d%Y')
squirrel_census_results_df['year'] = squirrel_census_results_df['date'].dt.year
squirrel_census_results_df['month'] = squirrel_census_results_df['date'].dt.month
squirrel_census_results_df['day'] = squirrel_census_results_df['date'].dt.day
squirrel_census_results_df['weekday'] = squirrel_census_results_df['date'].dt.day_name()

In [115]:
# Get all aboveground squirrels and manhattan trees
above_ground_df = squirrel_census_results_df[squirrel_census_results_df['location'] == 'Above Ground'].copy()
manhattan_trees_df = tree_census_results_df[tree_census_results_df['boroname']=='Manhattan'].copy()

# Coordinates of above-ground squirrels and trees 
squirrel_coords = above_ground_df[['x','y']].values
tree_coords = manhattan_trees_df[['latitude','longitude']].values

# Get GeoDataFrames
squirrels_gdf = gpd.GeoDataFrame(
    above_ground_df,
    geometry=gpd.points_from_xy(above_ground_df['x'], above_ground_df['y']), # x=lon, y=lat
    crs="EPSG:4326"
)
trees_gdf = gpd.GeoDataFrame(
    manhattan_trees_df,
    geometry=gpd.points_from_xy(manhattan_trees_df['longitude'], manhattan_trees_df['latitude']),
    crs="EPSG:4326"
)

# Project distances to meters 
squirrels_gdf = squirrels_gdf.to_crs(epsg=3857)
trees_gdf = trees_gdf.to_crs(epsg=3857)

# Get tree and squirrel coordinates 
tree_coords = np.array([(p.x, p.y) for p in trees_gdf.geometry])
squirrel_coords = np.array([(p.x, p.y) for p in squirrels_gdf.geometry])

# Get distances to nearest tree 
tree_kdtree = cKDTree(tree_coords)
distances, indices = tree_kdtree.query(squirrel_coords)

# Assign nearest tree ID and distance in meters
squirrels_gdf['nearest_tree'] = trees_gdf.iloc[indices]['tree_id'].values
squirrels_gdf['distance_meters'] = distances

# merge into dataset 
squirrel_census_results_df = squirrel_census_results_df.merge(
    squirrels_gdf[['unique_squirrel_id','nearest_tree','distance_meters']],
    on='unique_squirrel_id',
    how='left'
)

# Sort the DataFrame by distance in ascending order
squirrel_census_results_df.to_csv("squirrel_census_transformed.csv", index=False)